In [26]:
import pandas as pd
import numpy as np
import scipy as sp
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import patsy
import math
import json

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [27]:
movie_ratings = pd.read_csv("/afs/crc.nd.edu/user/j/jnazario/social_sensing_boxoffice/movieRatings.txt", names = ["Movie", "RT", "IMDB", "MC"])
movie_features = pd.read_json("/afs/crc.nd.edu/user/j/jnazario/social_sensing_boxoffice/tweetsMovies.json", typ='series')
#movie_popularity = pd.read_json("/afs/crc.nd.edu/user/j/jnazario/social_sensing_boxoffice/moviePopularity.json", typ='series')
#display(movie_features)

with open('/afs/crc.nd.edu/user/j/jnazario/social_sensing_boxoffice/boxEarnings.json', 'r') as f:
    data = json.load(f)

#movie_BOR = pd.read_json("data")
df = pd.DataFrame({'BOR': data})
df = df.reset_index()
df.columns = ["Movie","BOR"]
movie_BOR = df

In [28]:
movie_features = movie_features.T #transposed dataframe
movie_features = movie_features.reset_index()
movie_features.columns = ['Movie', 'feat'] #"sent","avg_rt"]
movie_features1 = pd.DataFrame(movie_features.feat.tolist(), columns=['sent', 'avg_rt'])

movie_features = movie_features[['Movie']]
#movie_features = movie_features.merge(movie_features[['Movie','sent','avg_rt']], on = ['Movie'])
movie_features = movie_features.join(movie_features1[['sent', 'avg_rt']])


In [29]:
display(movie_features)

,Movie,sent,avg_rt
0,Master Z: Ip Man Legacy,0.285000,1.022500
1,Spider-Man: Into the Spider-Verse,0.287500,1.588333
2,Aquaman,0.114167,0.306667
3,Fantastic Beasts: The Crimes of Grindelwald,0.900833,0.460000
4,Avengers: Infinity War,0.144167,0.737500
5,Mary Poppins Returns,0.409167,0.519167
6,Green Book,-0.233333,0.632500
7,Bohemian Rhapsody,0.202500,1.029167
8,Creed II,0.165833,0.165833
9,Black Panther,-0.267500,0.457500


In [30]:
#movie_BOR = movie_BOR[~movie_BOR['BOR'].isin([''])]
# BOR_int = movie_BOR['BOR'].astype(str).astype(int)
# BOR_int = pd.DataFrame(BOR_int)
# BOR_int.columns = ['BOR_int']
# display(movie_BOR)
# movie_BOR = movie_BOR.merge(BOR_int[['BOR_int']])

In [31]:
#result = pd.concat([movie_ratings, movie_features], axis=0, sort=False)
final = movie_ratings.merge(movie_features[['Movie','sent','avg_rt']], on = ['Movie'])
final = final.merge(movie_BOR[['Movie','BOR']], on = ['Movie'])
#final.dtypes
final
final = final[~final['BOR'].isin(['0'])]

final

,Movie,RT,IMDB,MC,sent,avg_rt,BOR
1,Spider-Man: Into the Spider-Verse,97.0,8.5,87.0,0.287500,1.588333,35363376
2,Aquaman,65.0,7.2,55.0,0.114167,0.306667,335061807
3,Fantastic Beasts: The Crimes of Grindelwald,37.0,6.7,52.0,0.900833,0.460000,62163104
4,Avengers: Infinity War,85.0,8.5,68.0,0.144167,0.737500,257698183
5,Mary Poppins Returns,79.0,7.0,66.0,0.409167,0.519167,23523121
6,Green Book,78.0,8.3,69.0,-0.233333,0.632500,320429
7,Bohemian Rhapsody,61.0,8.1,49.0,0.202500,1.029167,51061119
8,Creed II,84.0,7.3,66.0,0.165833,0.165833,35574710
9,Black Panther,97.0,7.3,88.0,-0.267500,0.457500,202003951
10,Ralph Breaks the Internet,88.0,7.2,71.0,0.338333,1.384167,56237634


<h1>Hard code box office revenue table</h1>

In [32]:
# d = {'Movie': ['Master Z: Ip Man Legacy', 'Spider-Man: Into the Spider-Verse', 'Aquaman', 'Fantastic Beasts: The Crimes of Grindelwald', 'Avengers: Infinity War', 'Mary Poppins Returns', 'Green Book'], 'BOR': [0.09, 35.4, 67.9, 62.2, 257.7, 23.5,0.32]}
# box_col = pd.DataFrame(data = d)
# box_col

In [33]:
# final = final.merge(box_col[['Movie','BOR']], on = ['Movie'])

<h1>Normalize Movie Ratings</h1>

In [34]:
# Normalize by Columns
final['IMDB']=(final['IMDB']-final['IMDB'].min())/(final['IMDB'].max()-final['IMDB'].min())
final['RT']=(final['RT']-final['RT'].min())/(final['RT'].max()-final['RT'].min())
final['MC']=(final['MC']-final['MC'].min())/(final['MC'].max()-final['MC'].min())
final['BOR']=(final['BOR']-final['BOR'].min())/(final['BOR'].max()-final['BOR'].min())

final

,Movie,RT,IMDB,MC,sent,avg_rt,BOR
1,Spider-Man: Into the Spider-Verse,0.979798,1.000000,0.8875,0.287500,1.588333,0.105093
2,Aquaman,0.656566,0.690476,0.4875,0.114167,0.306667,1.000000
3,Fantastic Beasts: The Crimes of Grindelwald,0.373737,0.571429,0.4500,0.900833,0.460000,0.185117
4,Avengers: Infinity War,0.858586,1.000000,0.6500,0.144167,0.737500,0.768990
5,Mary Poppins Returns,0.797980,0.642857,0.6250,0.409167,0.519167,0.069737
6,Green Book,0.787879,0.952381,0.6625,-0.233333,0.632500,0.000453
7,Bohemian Rhapsody,0.616162,0.904762,0.4125,0.202500,1.029167,0.151966
8,Creed II,0.848485,0.714286,0.6250,0.165833,0.165833,0.105724
9,Black Panther,0.979798,0.714286,0.9000,-0.267500,0.457500,0.602686
10,Ralph Breaks the Internet,0.888889,0.690476,0.6875,0.338333,1.384167,0.167424


In [35]:
final['sent'] = final['sent'].round()
final

,Movie,RT,IMDB,MC,sent,avg_rt,BOR
1,Spider-Man: Into the Spider-Verse,0.979798,1.000000,0.8875,0.0,1.588333,0.105093
2,Aquaman,0.656566,0.690476,0.4875,0.0,0.306667,1.000000
3,Fantastic Beasts: The Crimes of Grindelwald,0.373737,0.571429,0.4500,1.0,0.460000,0.185117
4,Avengers: Infinity War,0.858586,1.000000,0.6500,0.0,0.737500,0.768990
5,Mary Poppins Returns,0.797980,0.642857,0.6250,0.0,0.519167,0.069737
6,Green Book,0.787879,0.952381,0.6625,-0.0,0.632500,0.000453
7,Bohemian Rhapsody,0.616162,0.904762,0.4125,0.0,1.029167,0.151966
8,Creed II,0.848485,0.714286,0.6250,0.0,0.165833,0.105724
9,Black Panther,0.979798,0.714286,0.9000,-0.0,0.457500,0.602686
10,Ralph Breaks the Internet,0.888889,0.690476,0.6875,0.0,1.384167,0.167424


In [36]:
final = final[final.Movie != 'Aquaman']
final = final[final.Movie != 'Black Panther']
final = final[final.Movie != 'The Nun']
final = final[final.Movie != 'First Man']
final = final[final.Movie != 'The Favourite']
final = final[final.Movie != 'Green Book']
final

,Movie,RT,IMDB,MC,sent,avg_rt,BOR
1,Spider-Man: Into the Spider-Verse,0.979798,1.000000,0.8875,0.0,1.588333,0.105093
3,Fantastic Beasts: The Crimes of Grindelwald,0.373737,0.571429,0.4500,1.0,0.460000,0.185117
4,Avengers: Infinity War,0.858586,1.000000,0.6500,0.0,0.737500,0.768990
5,Mary Poppins Returns,0.797980,0.642857,0.6250,0.0,0.519167,0.069737
7,Bohemian Rhapsody,0.616162,0.904762,0.4125,0.0,1.029167,0.151966
8,Creed II,0.848485,0.714286,0.6250,0.0,0.165833,0.105724
10,Ralph Breaks the Internet,0.888889,0.690476,0.6875,0.0,1.384167,0.167424
11,Mortal Engines,0.272727,0.452381,0.3500,0.0,0.234167,0.022071
12,The Mule,0.707071,0.666667,0.5250,0.0,0.461667,0.051780
13,A Star Is Born,0.898990,0.833333,0.9000,0.0,1.814167,0.127621


<h1>Multiple Linear Regression Model </h1>

In [38]:
formula = "BOR ~ 1 + RT + IMDB + MC + sent + avg_rt"
y, X = patsy.dmatrices(formula, final, return_type='matrix')  #remember this drops missing data for you
model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                    BOR   R-squared:                       0.141
Model:                            OLS   Adj. R-squared:                  0.061
Method:                 Least Squares   F-statistic:                     1.773
Date:                Fri, 03 May 2019   Prob (F-statistic):              0.134
Time:                        17:31:17   Log-Likelihood:                 35.403
No. Observations:                  60   AIC:                            -58.81
Df Residuals:                      54   BIC:                            -46.24
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0233      0.056     -0.419      0.6

In [ ]:
#fig = plt.figure()
#fig.set_size_inches(18.5, 5.5, forward=True)
#plt.scatter(X,y)

<h1>Plot data and regression model</h1>

In [ ]:
import seaborn as sns
# Use a Linear Model Plot
sns.lmplot(x='sent', y='BOR',data=final)

<h1>Residuals Plot</h1>

In [ ]:
infl = results.get_influence() #To get the residuals
plt.scatter(results.fittedvalues,infl.resid_studentized_internal)

<h1>Neural Networks</h1>

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
scaler = StandardScaler()
# Fit only to the training data
scaler.fit(X_train)
StandardScaler(copy=True, with_mean=True, with_std=True)
# Now apply the transformations to the data:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
reg = MLPRegressor(hidden_layer_sizes=(100,),  activation='relu', solver='adam', alpha=0.001, batch_size='auto',
    learning_rate='constant', learning_rate_init=0.01, power_t=0.5, max_iter=1000, shuffle=True,
    random_state=None, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True,
    early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
reg = reg.fit(X_train,y_train)
#reg.score(X_train, y_train, sample_weight=None)
y_test

In [ ]:
predictions = reg.predict(X_test)
# create Pandas Series with define indexes 
pred = pd.Series(predictions) 
myP = pd.Series.to_frame(pred)

# create Pandas Series with define indexes 
myArray = np.ndarray.tolist(y_test)
myList = []
for i in myArray:
    myList.append(i.pop())
y_t = pd.Series(myList) 
myT = pd.Series.to_frame(y_t)

In [ ]:
# d = {'predictions': myP, 'y_test': myT}
# pd.DataFrame(data=d, index=0)
myP.columns = ['pred']
myT.columns = ['actual']
testing = myP.join(myT[['actual']])

testing

In [ ]:
r2_score(testing['actual'], testing['pred'])